# Fortsetzung Selenium

In der letzten Woche haben wir uns angesehen, wie wir Zitate von der Seite https://quotes.toscrape.com/js/ mithilfe von Selenium scrapen können. Dabei haben wir Selenium in der Version 3 und den Python webdriver-manager verwendet. Wir haben uns dazu ein Beispielskript von Lewis Kori angesehen, das ebenfalls Selenium 3-Funktionen verwendet. Im Internet findet man zudem viele Tutorials und Beispielskripte, die Selenium 3 verwenden. In dieser Stunde werden wir aber nun doch auf Selenium 4 umsteigen. Denn in Selenium ab der Version 4.6 gibt es einige Verbesserungen: Es gibt zum Beispiel einen eigenen Selenium Manager, der den Python webdriver-manager ersetzt. Daneben gibt es Möglichkeiten, vor dem Aufruf der Seite einen geografischen Standort anzugeben. So kann eine Webseite in verschiedenen Versionen aufgerufen werden.  Leider haben sich auch einige Funktions- bzw. Methodennamen in Selenium 4 gegenüber Selenium 3 geändert, zum Beispiel die Methode find_element_by_class_name(), die wir bereits verwendet haben.

### Upgrade von Selenium 3 auf 4

Das Upgrade auf Selenium 4 birgt allerdings einige Fallstricke, weswegen ich mich zunächst dagegen entschieden hatte: Zum einen kann es sein, dass Selenium 4 bei euch langsamer ist als Selenium 3. Zum anderen kann der Selenium Manager aktuell noch nicht in der Anaconda-Distribution von Selenium 4 verwendet werden (siehe dazu [diese Diskussion](https://github.com/SeleniumHQ/selenium/issues/11234)). Und zuletzt kann es sein, dass ihr zunächst Anaconda aktualisieren müsst, um Selenium 4 installieren zu können. Um Anaconda zu aktualisieren, müsst ihr über das Terminal / Command Prompt zunächst den Befehl `conda update conda` ausführen, danach könnt ihr die virtuelle Umgebung aktivieren und Jupyter Lab wieder wie gewohnt starten.

Um von Selenium 3 auf Selenium 4 umzusteigen, muss leider zunächst den webdriver-manager wieder deinstalliert werden. Das geht mit dem folgenden Code:

In [ ]:
# import sys
# !conda remove --yes --prefix {sys.prefix} webdriver-manager

Jetzt könnt ihr versuchen, direkt Selenium 4 zu installieren. Da es wie bereits erwähnt ein Problem bei der Verwendung von Selenium Manager in der Anaconda-Distribution von Selenium 4 gibt, installieren wir Selenium 4 ausnahmsweise via pip, den Standard-Paketmanager für Python-Pakete. Das Ausrufezeichen vor pip braucht ihr, wenn ihr pip aus dem Jupyter Notebook heraus verwendet:

In [ ]:
# !pip install selenium==4.6.0

Jetzt solltet ihr überprüfen, ob webdriver-manager erfolgreich deinstalliert und Selenium 4 installiert wurde:

In [ ]:
# !conda list # wurden die Pakete aktualisiert?
# selenium.__version__ # Selenium-Version überprüfen

Wenn in der Liste immer noch Selenium 3 steht, müsst ihr zunächst Selenium 3 deinstallieren, und dann Selenium 4 via pip installieren:

In [ ]:
# !conda remove --yes --prefix {sys.prefix} selenium

Manchmal gibt es ein Problem mit der Installation von Paketen via pip im Zusammenhang mit Anaconda. Falls das bei euch der Fall sein sollte, schaut euch diese Lösungsstrategien an: https://stackoverflow.com/questions/41060382/using-pip-to-install-packages-to-anaconda-environment.


### Ortsangaben zu Unterkünften von airbnb.com scrapen

Bei der Bearbeitung des Übungsblatts zur heutigen Stunde habt ihr Ortstangaben zu Unterkünften auf der Startseite von https://www.airbnb.com/ extrahiert. Die Lösung mit Selenium 3 sah bei euch wahrscheinlich in etwa so aus:

In [ ]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# import time

# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get("https://www.airbnb.com/")
# time.sleep(5)
# unterkuenfte = driver.find_elements_by_class_name("t1jojoys.dir.dir-ltr") #t1jojoys dir dir-ltr
# unterkuenfte_orte = [unterkunft.text for unterkunft in unterkuenfte]
# unterkuenfte_orte

Jetzt reproduzieren wir die Aufgabe mit Selenium 4. Beachtet, dass die Methode find_element_by_class_name("name_der_klasse") in Selenium 4 ersetzt wurde durch find_elements(By.CLASS_NAME, "name_der_klasse"). Beachtet auch, dass anstelle des Python webdriver-managers jetzt der  Selenium Manager verwendet wird, sodass die Funktion webdriver.Chrome() ohne Argument aufgerufen wird. Alles andere ist genau so wie vorher.

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.selenium_manager import SeleniumManager
# from selenium.webdriver.common.by import By
# import time

# driver = webdriver.Chrome()
# driver.get("https://www.airbnb.com/")
# time.sleep(5) # hier reichen 5 Sekunden eventuell nicht
# unterkuenfte = driver.find_elements(By.CLASS_NAME, "t1jojoys.dir.dir-ltr")
# unterkuenfte_orte = [unterkunft.text for unterkunft in unterkuenfte]
# unterkuenfte_orte

Unser Ortsnamen-Scraper hat aber nicht alle Ortsangaben extrahiert, sondern nur die ersten 20. Woran liegt das? Die Seite verwendet infinite scrolling; Seiteninhalte werden also erst geladen, wenn auf der Seite heruntergescrollt wird.

### Infinite Scrolling simulieren

Wir müssen also zunächst das Durchscrollen der Seite simulieren, damit alle Inhalte geladen werden. Dazu können wir wieder den Code von jemand anderem verwenden, denn das Scrolling funktioniert immer genau gleich. Wir richten uns nach dem Blogbeitrag von [Kuan Wei](https://medium.com/analytics-vidhya/using-python-and-selenium-to-scrape-infinite-scroll-web-pages-825d12c24ec7):

In [ ]:
# scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
# screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
# i = 1

Im Code oben wird zunächst im Browser das JavaScript Code-Snippet return window.screen.height ausgeführt, um  die Höhe des Bildschirms, auf dem das aktuelle Browserfenster angezeigt wird, in Pixeln abzurufen. Bei window.screen handelt es sich um ein JavaScript-Objekt, bei .height um ein Attribut des Objekts window.screen, und return ist das JavaScript-Pendant zur return-Anweisung in Python, die in Funktionsaufrufen verwendet wird, um einen Wert zurückzugeben. Wie genau diese Werte extrahiert werden, müssen wir nicht unbedingt wissen, um den Code zu verwenden. Aber wenn sich jemand nähergehend damit beschäftigen möchte, empfehle ich diese Seite: https://www.webtechnologien.com/advanced-tutorials/javascript-bom/.

In [ ]:
# screen_height

In [ ]:
# while True:
#     # scroll one screen height each time
#     driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
#     i += 1
#     time.sleep(scroll_pause_time)
#     # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
#     scroll_height = driver.execute_script("return document.body.scrollHeight;")
#     # Break the loop when the height we need to scroll to is larger than the total scroll height
#     if (screen_height) * i > scroll_height:
#         break

Was macht die while-Schleife? Es wird wieder auf die Methode execute_script() des Webdriver-Objekts driver zurückgegriffen, um ein JavaScript-Code-Snippet auszuführen. Danach wird die Zählvariable i erhöht und die mit der Variable scroll_pause_time festgelegte Wartezeit eingeleitet, und anschließend wird ein weiteres JavaScript-Code-Snippet ausgeführt. Das erste JavaScript Code Snippet führt das eigentliche Scrollen aus, wobei in jedem Schleifendurchlauf genau so weit gescrollt wird, wie der Bildschirm hoch ist (= screen_height): Im ersten Durchlauf bis screen_height, im zweiten Durchlauf bis 2*screen_height, usw. Das zweite Snippet ruft die Höhe des scrollbaren Seiteninhalts auf, also sowohl der sichtbare als auch der noch unsichtbare, durch Scrollen erreichbare Seiteninhalt.
Zuletzt wird überprüft, ob im nachfolgenden Schleifendurchlauf (also mit i+=1) der geplante zu scrollende Bereich größer ist als der insgesamt scrollbare Seiteninhalt. Wenn dies der Fall ist, wird die Schleife terminiert.

Wenn die while-Schleife terminiert, ist der gesamte Seiteninhalt durchscrollt und gerendert. Anschließend können wieder die Ortsangaben extrahiert werden: find_elements() findet jetzt nicht nur die ersten 20 Suchergebnisse, sondern alle Ergebnisse.

In [ ]:
# unterkuenfte = driver.find_elements(By.CLASS_NAME, "t1jojoys.dir.dir-ltr")
# unterkuenfte_orte = [unterkunft.text for unterkunft in unterkuenfte]
# unterkuenfte_orte

Zuletzt schließen wir das aktuelle Browserfenster und die Session, also die Sitzung, welche durch den Aufruf des Chrome Webdrivers gestartet wird:

In [ ]:
# driver.quit()

### Suche benutzen, Mausklick und Tastatureingabe simulieren

Als nächstes sehen wir uns an, wie mithilfe von Selenium 4 die Suchmaske auf airbnb.com verwendet werden kann und, wie Buttons via Mausklick betätigt und eine Tastatureingabe getätigt werden können.

Zunächst starten wir wieder den Webdriver und senden eine Anfrage für die Seite https://www.airbnb.com/.

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.selenium_manager import SeleniumManager
# from selenium.webdriver.common.by import By
# import time

# driver = webdriver.Chrome()
# driver.get("https://www.airbnb.com/")

Als nächstes wollen wir nach Unterkünften in Berlin suchen. Dazu führen wir die Suche einmal in unserem regulären Chrome Browser aus, um herauszufinden, mit welchen Bestandteilen des User Interfaces bei der Suche interagiert werden muss. Als erstes muss auf den Suchbutton geklickt werden:

:::{figure-md}
<img src="beispiel_airbnb_1.png" alt="Airbnb Beispiel" class="bg-transparent" width="75%">

Abb. 1: Beispiel Airbnb
:::

Um den Mausklick auf den Button zu simulieren, müssen wir erst das Element finden und anschließend mithilfe der Methode .click() den Mausklick simulieren. In diesem Fall suchen wir das Element mithilfe des class-Attributs, weil wir mithilfe einer Suche im Quelltext (Strg+F) leicht feststellen können, dass es kein weiteres Element mit dem Attribut class="s19wqnbx dir dir-ltr" gibt.

In [ ]:
# Auf den Suchbutton klicken: Sonst ist das Suchfeld nicht sichtbar
# driver.find_element(By.CLASS_NAME, "s19wqnbx.dir.dir-ltr").click()

Nach dem Klick auf den Suchbutton öffnet sich ein Suchfeld, in dem wir unseren Suchbegriff eingeben können. Im regulären Chrome-Browser können wir, wieder mithilfe der Entwicklertools, feststellen, dass das Suchfeld über ein HTML-input-Element dargestellt wird:

:::{figure-md}
<img src="beispiel_airbnb_2.png" alt="Airbnb Beispiel" class="bg-transparent" width="75%">

Abb. 2: Beispiel Airbnb
:::

Um einen Suchbegriff eingeben zu können, muss das input-Element zunächst gefunden werden. In diesem Fall hat das gesuchte HTML-Element nicht nur ein Attribut class, sondern auch ein Attribut id mit dem Wert, "bigsearch-query-location-input", welches erlaubt, das Element eindeutig zu identifizieren. Zur Suche können wir nun entweder find_element(By.ID, "id_des_elements") oder find_element(By.XPATH, "xpath_ausdruck") verwenden. XPath ist eine sogenannte Pfadbeschreibungssprache, die zur Suche in XML- und HTML-Dokumenten verwendet wird. Um XPath-Ausdrücke zu verwenden, müssen wir uns aber nicht mit XPath auskennen, denn auch dabei helfen die Browser-Entwicklertools. Der XPath-Ausdruck, der den Pfad zu einem bestimmten Element beschreibt, kann ganz einfach mit Rechtsklick auf ein Element und die Option Copy -> Copy XPath kopiert werden.

:::{figure-md}
<img src="beispiel_airbnb_3.png" alt="Airbnb Beispiel" class="bg-transparent" width="75%">

Abb. 3: Beispiel Airbnb
:::

Der XPath zum gesuchten input-Element ist //*\[@id="bigsearch-query-location-input"\]. Das * steht für ein beliebiges HTML-Element, aber wir können auch den Namen des HTML-Elements einsetzen, um bei vielen XPath-Ausdrücken den Überblick zu behalten:

In [ ]:
# Input-Element finden, in das die Suchbgegriffe eingegeben werden können
# suchfeld = driver.find_element(By.XPATH, "//input[@id='bigsearch-query-location-input']")

Beachtet, dass im Code oben die inneren Anführungszeichen angepasst wurden, um sie von den doppelten äußeren Anführungszeichen zu unterscheiden. Das ist unbedingt notwendig, weil sonst der XPath-Ausdruck nicht richtig interpretiert werden kann.

Wenn das Element gefunden ist, kann es mithilfe der Methode send_keys() zur Eingabe eines Suchbegriffs addressiert werden.

In [ ]:
# Suchbegriff eingeben
# suchfeld.send_keys("Berlin")

Die Suche muss anschließend noch durch Betätigung der Enter-Taste bestätigt werden:

In [ ]:
# Tasteneingabe ENTER
# from selenium.webdriver.common.keys import Keys
# suchfeld.send_keys(Keys.ENTER)

Durch Bestätigung der Suche mit Enter wird automatisch ein Fenster zur Auswahl eines Reisetermins geöffnet. Hier wollen wir die Option "flexible" auswählen. Dazu müssen wir zunächst wieder das gesuchte Element identifizieren:

:::{figure-md}
<img src="beispiel_airbnb_4.png" alt="Airbnb Beispiel" class="bg-transparent" width="75%">

Abb. 4: Beispiel Airbnb
:::

Das gesuchte HTML-Element hat wieder eine ID, über die es eindeutig identifiziert werden kann. Diesmal verwenden wir find_element(By.ID, "id_des_elements"), damit ihr diese Verwendung der find_element-Methode auch einmal gesehen habt. Die Id könnt ihr einfach aus den Browser-Entwicklertools mit Doppelklick auf das Id-Attribut kopieren.

In [ ]:
# Zeit aussuchen: Flexible
# driver.find_element(By.ID, "tab--tabs--2").click()

Zuletzt müssen wir unsere Suche noch mit Klick auf den Suchbutton bestätigen. In diesem Fall wird über Rechtsklick auf den Suchbutton und Auswahl der Option "Inspect" allerdings nicht ganz das richtige Element gefunden: Gefunden wird das span-Element mit dem Attribut class="t1dqvypu dir dir-ltr"; gesucht haben wir aber eigentlich den gesamten Suchbutton, also das button-Element mit dem Attribut class="brqqy3t bd1b9vv dir dir-ltr". Bei der Verwendung von "Inspect" ist also immer Mitdenken erforderlich, denn nicht immer wird ganz genau das Element getroffen, das gesucht wird.

:::{figure-md}
<img src="beispiel_airbnb_5.png" alt="Airbnb Beispiel" class="bg-transparent" width="75%">

Abb. 5: Beispiel Airbnb
:::

Um den Suchbutton zu klicken, überprüfen wir erst, ob die Klasse "brqqy3t bd1b9vv dir dir-ltr" noch einmal verwendet wird. Dies ist nicht der Fall, sodass wir das class-Attribut auch in diesem Fall zur eindeutigen Suche nach dem button-Element verwenden können:

In [ ]:
# Auf den Suchbutton klicken: Suche bestätigen
# driver.find_element(By.CLASS_NAME, "brqqy3t.bd1b9vv.dir.dir-ltr").click() # Punkte statt Leerzeichen

Zuallerletzt führen wir wieder den Code aus der Übungsaufgabe aus, um alle Ortsangaben von den ersten 20 vorgeladenen Suchergebnissen zu extrahieren, und beenden die Session:

In [ ]:
# Orte extrahieren
# unterkuenfte = driver.find_elements(By.CLASS_NAME, "t1jojoys.dir.dir-ltr")
# unterkuenfte_orte = [unterkunft.text for unterkunft in unterkuenfte]
# unterkuenfte_orte

In [ ]:
# driver.quit()

### Quellen

```{bibliography}
   :list: enumerated
   :filter: keywords % "sel_2"
```